In [ ]:
import os
import numpy as numpy
import pandas as pd
import plotly.express as px
import geopandas as gpd
import shapely.geometry
import wget

In [8]:
temp = pd.read_csv(r".\data\bus_routes.csv", encoding='cp949')
temp.head()

,업체명,업종,노선명,노선유형,노선상태,운행형태,변경인가일자,최초인가일자,운송개시일,인가대수,...,총인가거리(편도),굴곡도,중간배차유무,벽지노선,기점정류소,종점정류소,관할관청,업체아이디,노선아이디,데이터기준일자
0,(주)용남고속버스라인,시내버스,3003,경기도 공공버스,정상,직행좌석형시내버스,2020-04-10,NaN,2021-01-01,14,...,32.5,1.39,N,N,능실마을21단지.수원여대입구,강남역나라빌딩앞,경기도 수원시,4105000,200000108,2024-08-29
1,(주)용남고속버스라인,시내버스,3003예약,일반,정상,직행좌석형시내버스,2020-04-06,2020-03-30,2020-04-02,2,...,32.5,NaN,N,N,삼환나우빌아파트,선바위역,경기도 수원시,4105000,200000293,2024-08-29
2,(주)용남고속버스라인,시내버스,7002,경기도 공공버스,정상,직행좌석형시내버스,2019-05-13,NaN,2021-04-05,6,...,43.3,1.49,N,N,수원남부버스공영차고지,인덕원역4호선,경기도 수원시,4105000,200000109,2024-08-29
3,경진여객운수(주),시내버스,3000,경기도 공공버스,정상,직행좌석형시내버스,2020-05-07,NaN,2021-03-01,17,...,37.7,1.39,N,N,능실마을21단지.수원여대입구,강남역나라빌딩앞,경기도 수원시,4110800,200000104,2024-08-29
4,경진여객운수(주),시내버스,3000(예약),일반,정상,직행좌석형시내버스,NaN,2020-03-27,2020-04-01,2,...,27.9,1.26,N,N,수원종합운동장.수원시체육회관.아너스빌위즈,강남역나라빌딩앞,경기도 수원시,4110800,200000283,2024-08-29


In [1]:
# https://business.juso.go.kr/addrlink/elctrnMapProvd/geoDBDwldList.do?menu=%EA%B5%AC%EC%97%AD%EC%9D%98%20%EB%8F%84%ED%98%95
# https://sksdudtjs.tistory.com/23
# https://www.data.go.kr/data/15050416/fileData.do



### 버스 정류장 위치 정보 전처리

In [ ]:
import os
import pandas as pd

In [5]:
df = pd.read_csv(r".\data\bus_stop_loc\국토교통부_전국 버스정류장 위치정보_20241028.csv", encoding='utf-8')
df

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명
0,ADB354000001,길안정류장,36.458658,128.891228,2024-10-28,540001.0,37040,경상북도 안동시,안동BIS
1,ADB354000002,고란.계명산휴양림입구,36.397708,128.924029,2024-10-28,540002.0,37040,경상북도 안동시,안동BIS
2,ADB354000003,금곡리,36.397504,128.926821,2024-10-28,540003.0,37040,경상북도 안동시,안동BIS
3,ADB354000004,금소리1,36.509449,128.849671,2024-10-28,540004.0,37040,경상북도 안동시,안동BIS
4,ADB354000005,금소리5,36.503629,128.854783,2024-10-28,540005.0,37040,경상북도 안동시,안동BIS
...,...,...,...,...,...,...,...,...,...
206017,YEB328012050,중마시장,34.936978,127.697847,2024-10-28,0.0,36020,전라남도 여수시,여수BIS
206018,YEB328013028,사곡,34.961730,127.628080,2024-10-28,3283028.0,36020,전라남도 여수시,여수BIS
206019,YEB328013038,북부정류장,34.979260,127.582940,2024-10-28,3283038.0,36020,전라남도 여수시,여수BIS
206020,YEB328013041,매화아파트,34.983380,127.580570,2024-10-28,3283041.0,36020,전라남도 여수시,여수BIS


In [ ]:
suwon_data = df[df["도시명"] == "경기도 수원시"]
print(suwon_data)

               정류장번호              정류장명         위도          경도       정보수집일  \
64944   GGB200000174  상광교신다리.광교로컬푸드직매장  37.327667  127.014917  2024-10-28   
64945   GGB200000175             용머리다리  37.323750  127.015467  2024-10-28   
64946   GGB200000176              용머리앞  37.321183  127.015567  2024-10-28   
64947   GGB200000177               장안문  37.290133  127.015433  2024-10-28   
64948   GGB200000178        KB손해보험인재니움  37.291767  127.017967  2024-10-28   
...              ...               ...        ...         ...         ...   
108708  GGB277103124         벌터교차로(경유)  37.262100  126.996300  2024-10-28   
108709  GGB277103125       환승센터출입1(경유)  37.268417  126.997850  2024-10-28   
108710  GGB277103126       환승센터출입2(경유)  37.271183  126.996683  2024-10-28   
108711  GGB277103127        세평지하차도(경유)  37.262383  127.003033  2024-10-28   
108712  GGB277103128         탑동사거리(경유)  37.274000  126.969017  2024-10-28   

        모바일단축번호   도시코드      도시명  관리도시명  
64944    1152.0  31010  경기도 수원시  경

In [1]:
## 경기도 버스 정류소 현황
import pandas as pd
df = pd.read_csv(r".\data\노드\bus_stop_loc\버스정류소현황.csv", encoding='utf-8')
df.head()

,시군명,정류소명,정류소영문명,정류소id,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도
0,가평군,3반상회앞,3-ban Market,239000307,44655.0,노변정류장,경기도 가평군,경기도 가평군 가평읍,37.811250,127.521467
1,가평군,가는골,Ganeungol,239000874,44847.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.967533,127.418617
2,가평군,가는골,Ganeungol,239000875,44848.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.967450,127.418650
3,가평군,가둘기,Gadulgi,239000061,44280.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.900067,127.511583
4,가평군,가둘기(목동방면),Gadulgi,239000459,44354.0,노변정류장,경기도 가평군,경기도 가평군 북면,37.900033,127.511700


In [ ]:
suwon_df = df[df["관할관청"].str.contains("경기도 수원", na=False)]
suwon_df

0        False
1        False
2        False
3        False
4        False
         ...  
34580    False
34581    False
34582    False
34583    False
34584    False
Name: 관할관청, Length: 34585, dtype: bool